In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fine-tune-books/fine_tuning_df.csv
/kaggle/input/fine-tune-books/requirements.txt


In [2]:
!pip install -r /kaggle/input/fine-tune-books/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

2025-06-30 09:04:48.254883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751274288.440387      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751274288.492220      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANBD")
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manishkundu21 (manishkundu21-iit-kharagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding, # ADDED: For dynamic padding
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import gc

def compute_metrics(p):
    """
    Computes and returns a dictionary of metrics for evaluation.
    
    Args:
        p (EvalPrediction): An object containing the model's predictions and true labels.

    Returns:
        dict: A dictionary containing accuracy, f1-score, precision, and recall.
    """
    # Extract predictions and labels
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    
    # Calculate metrics using the correct function name
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


def main():
    """
    Main function to run the fine-tuning process.
    """

    print("Loading and preparing data...")
    df = pd.read_csv('/kaggle/input/fine-tune-books/fine_tuning_df.csv')
    df = df.rename(columns={"desc":"text", "simple_categories":"label"})

    # Map text labels to integers
    label2id = {"Nonfiction": 0, "Fiction": 1}
    id2label = {0: "Nonfiction", 1: "Fiction"}
    df['label'] = df['label'].map(label2id)

    # Split the data into training and validation sets
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    # Convert pandas DataFrames to Hugging Face Dataset objects
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    print(f"Data loaded. Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


    # Load Tokenizer and Preprocess Data
    model_name = "microsoft/deberta-v3-base"
    print(f"Loading tokenizer for model: {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_function(examples):
        """
        Tokenizes the text data.
        We will only truncate, and the DataCollator will handle padding dynamically.
        """
        return tokenizer(examples["text"], truncation=True, max_length=512)

    print("Tokenizing datasets...")
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

    # Remove original text column and set format for PyTorch
    tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text", "__index_level_0__"])
    tokenized_val_dataset = tokenized_val_dataset.remove_columns(["text", "__index_level_0__"])

    # Initialize the DataCollator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    # Load the Model
    print("Loading pre-trained model for sequence classification...")
    # Clean up memory before loading the model
    gc.collect()
    torch.cuda.empty_cache()

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        id2label=id2label,
        label2id=label2id
    )
    
    # Check if a GPU is available and move the model to the GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Model loaded on: {device}")

    # Define Training Arguments for Speed and Memory Efficiency
    print("Defining training arguments...")
    training_args = TrainingArguments(
        output_dir="./deberta-fiction-nonfiction-classifier",
        num_train_epochs=3,
        per_device_train_batch_size=2,        
        per_device_eval_batch_size=8,         
        gradient_accumulation_steps=2,       
        gradient_checkpointing=True,         
        fp16=True,                            
        dataloader_num_workers=4,            
        warmup_steps=50,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,                    
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    # Initialize the Trainer
    print("Initializing Trainer...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator, 
    )

    # Start Fine-Tuning
    print("Starting fine-tuning...")
    trainer.train()
    print("Fine-tuning finished.")

    # Evaluate the Best Model
    print("Evaluating the best model on the validation set...")
    eval_results = trainer.evaluate()
    print("Evaluation results:")
    print(eval_results)
    
    # Save the Fine-Tuned Model and Tokenizer
    final_model_path = "./final_model"
    print(f"Saving the final model to {final_model_path}...")
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    print("Model and tokenizer saved successfully.")

if __name__ == "__main__":
    main()

Loading and preparing data...
Data loaded. Training samples: 3150, Validation samples: 788
Loading tokenizer for model: microsoft/deberta-v3-base...


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Tokenizing datasets...


Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/788 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.594900,0.520759,0.791878,0.000000,0.000000,0.000000
2,0.260000,0.341959,0.918782,0.792208,0.847222,0.743902


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Fine-tuning finished.
Evaluating the best model on the validation set...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Evaluation results:
{'eval_loss': 0.3419589400291443, 'eval_accuracy': 0.9187817258883249, 'eval_f1': 0.7922077922077924, 'eval_precision': 0.8472222222222222, 'eval_recall': 0.7439024390243902, 'eval_runtime': 12.9647, 'eval_samples_per_second': 60.781, 'eval_steps_per_second': 7.636, 'epoch': 2.996825396825397}
Saving the final model to ./final_model...
Model and tokenizer saved successfully.
